In [ ]:
Código para descarga de valoraciones y opiniones de retail de trustedshops.es

In [ ]:
import requests
from bs4 import BeautifulSoup
import re

In [ ]:
#Obtener listado de categorías

categories = []
url = "https://www.trustedshops.es"
soup = BeautifulSoup(requests.get(url+"/shops/").content, 'html.parser')
for l in soup.find(class_="CategoryFilterstyles__Category-vu79ja-1 eDJvma").find_all("li"):
    categories.append(l.a.get('href'))
print(categories)
print("Número de categorías: ", len(categories))
print("-------CATEGORÍAS OBTENIDAS--------")

In [ ]:
import pandas as pd
import glob

In [ ]:
#Obtener listado de empresas; para cada una se obtendrán lás páginas de comentarios.

categories=['/shops/moda/', '/shops/muebles_y_decoracion/', '/shops/ocio_y_coleccionismo/']
index=0
for c in categories:
    companies = []
    i=1
    soup = BeautifulSoup(requests.get(url+c+"?page="+str(i)).content, 'html.parser')
    for e in soup.find_all(class_="CardFooterstyles__CardFooter-sc-1wr2qx-0 eTBAbH"):
            companies.append(e.a.get('href'))
    while soup.find(class_="Iconstyles__Icon-sc-1aiy086-0 bqecvv hls hls-icon-arrow-chevron-right"):
        i+=1
        soup = BeautifulSoup(requests.get(url+c+"?page="+str(i)).content, 'html.parser')
        for g in soup.find_all(class_="CardFooterstyles__CardFooter-sc-1wr2qx-0 eTBAbH"):
            companies.append(g.a.get('href'))    
    print("...obtenidas empresas categoría: ", c, " , con número de empresas: ", len(companies))
    
    #Obtener comentarios de cada página de cada empresa

    reviews = []
    ratings = []
    
    #companies =["https://www.trustedshops.es/evaluacion/info_XC3EA2E3A7F3D4EEC360AE42D0EA647F4.html","https://www.trustedshops.es/evaluacion/info_X1F21D233D0237DD6D2662A63A57B402B.html" ]
    
    for m in companies:
    #c="https://www.trustedshops.es/evaluacion/info_XC3EA2E3A7F3D4EEC360AE42D0EA647F4.html"
        i=1
        soup = BeautifulSoup(requests.get(m+"?page="+str(i)).content, 'html.parser')
        check = soup.find(class_="ng-tns-c118-5 main-box no-margin mb-3 ng-star-inserted")
        if (check):
            post_time = check.select("span[class='ng-star-inserted']")[0].text    #guardo la hora del primer post para comprobar luego si está repitiendose
        else:
            print("0 páginas de reseñas extraidas de compañia con url: ", m)
            continue
        for r in soup.find_all(class_="ng-tns-c118-5 main-box no-margin mb-3 ng-star-inserted"):
            stars = r.find(class_="star tsproi tsproi-star-filled active ng-star-inserted")
            stars_count = 0
            while str(type(stars)) == "<class 'bs4.element.Tag'>":
                if stars.get('class')[3] == 'active':
                    stars_count+=1
                stars = stars.next_sibling
            #print(stars_count)
            if r.select('div[class^=ng-tns-c72-]'):        #comprobar que valoraciones sin comentario
                ratings.append(stars_count)
                reviews.append(r.select('div[class^=ng-tns-c72-]')[0].text.strip())
                #print(r.select('div[class^=ng-tns-c72-]')[0].text)
        while soup.find(class_="col-auto pr-0 pl-0 next").get('style') == "display:;":       #comprobar si hay más páginas mediante la visibilidad de la flecha 'next'
            print("página de reseñas: ", i, end="\r")
            i+=1
            soup = BeautifulSoup(requests.get(m+"?page="+str(i)).content, 'html.parser')
        
            #comprobación paara terminar, a veces no salta con el display:none y repite la primera página infinito
            if not soup.find(class_="ng-tns-c118-5 main-box no-margin mb-3 ng-star-inserted"):
                break
            if soup.find(class_="ng-tns-c118-5 main-box no-margin mb-3 ng-star-inserted").select("span[class='ng-star-inserted']")[0].text == post_time:
                break
            for r in soup.find_all(class_="ng-tns-c118-5 main-box no-margin mb-3 ng-star-inserted"):
                stars = r.find(class_="star tsproi tsproi-star-filled active ng-star-inserted")
                stars_count = 0
                while str(type(stars)) == "<class 'bs4.element.Tag'>":
                    if stars.get('class')[3] == 'active':
                        stars_count+=1
                    stars = stars.next_sibling
                #print(stars_count, end="\r")        
                if r.select('div[class^=ng-tns-c72-]'):        #comprobar que valoraciones sin comentario
                    reviews.append(r.select('div[class^=ng-tns-c72-]')[0].text.strip())
                    ratings.append(stars_count)
                    #print(r.select('div[class^=ng-tns-c72-]')[0].text)
        print(i," páginas de reseñas extraidas de compañia con url: ", m)
        print("reseñas: ", len(reviews), "valoraciones: ", len(ratings))
        
    #Guardar comentarios por categorías en archivo excel 

    fname = "trustedshops/comentarios_" + (categories[index].split('/')[2]) + ".xlsx"
    df = pd.DataFrame({'comentarios':reviews, 'valoraciones':ratings})
    with pd.ExcelWriter(fname) as writer:
        df.to_excel(writer, encoding='utf-8-sig')

    len_ratings=len(ratings)
    print("reseñas totales de categoría", (categories[index].split('/')[2]), ": ", len(reviews))
    print("valoraciones totales de categoría", (categories[index].split('/')[2]), ": ", len_ratings)
    
    index+=1

print("----------EXTRACCIÓN RESEÑAS TERMINADO---------------")


In [ ]:
ANEXO: Leer y evaluar Sentiment de todos los comentarios guardados (en lugar de evaluar en cada categoria)

In [ ]:

#Crear dataframe, obtener el sentiment y guardarlo en excel comparado con la valoración del usuario
# Modo append en el df de todos los excel, luego a listas y luego a df y excel 

files = glob.glob("trustedshops/*.xlsx")
df = pd.DataFrame()
for fl in files:
    excel = pd.read_excel(fl)
    df = df.append(excel)
reviews = df['comentarios'].tolist()
ratings = df['valoraciones'].tolist()
df = pd.DataFrame()
file_name='pruebas_sentiment_scrapping'+(categories[index].split('/')[2])+'.xlsx'
headers = {'Authorization' : '***aquí va el token de autorización***9'}

for t,s in zip(reviews, ratings):
    payload = {'texto': t}
    r = requests.post('http://***aquí la ip del endpoint del sentiment***/',data=payload,headers=headers)
    result = r.json()
    new_sentiment = {'new_sentiment': (s - 3)}            # ajustar del rango 1...5 a -2...2
    result.update(new_sentiment)
    df = df.append(result, ignore_index=True)
    print(s)
df.to_excel(file_name, encoding='utf-8-sig')

print("--------TERMINADO---------")